In [1]:
!pip install pytesseract

In [2]:
!pip install pypdf2
!pip install pdf2image

In [3]:
!pip install tools

In [4]:
!pip install dateparser

In [5]:
!pip install PyMuPDF


In [6]:
!pip install ollama


In [12]:
import pytesseract



In [16]:
import fitz  # PyMuPDF
import re
import json
import ollama
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import os

# Step 1: Extract text from PDF or Image
def extract_text(file_path):
    text = ""

    # Check file extension
    _, ext = os.path.splitext(file_path.lower())

    if ext in [".pdf"]:
        doc = fitz.open(file_path)
        for page in doc:
            text += page.get_text()

        if not text.strip():
            images = convert_from_path(file_path)
            for img in images:
                text += pytesseract.image_to_string(img)

    elif ext in [".jpg", ".jpeg", ".png"]:
        img = Image.open(file_path)
        text = pytesseract.image_to_string(img)

    else:
        raise ValueError("Unsupported file format. Please upload a PDF or an Image (jpg, png).")

    return text
pytesseract.pytesseract.tesseract_cmd = r"D:\Tesseract-OCR\tesseract.exe"
# Step 2: Send text to Ollama (Mistral Model)
def query_llm(text):
    prompt = f"""
You are a professional resume parser.

Given the following RESUME TEXT:

{text}

Extract and output a JSON with the following fields:

{{
  "name": "",
  "email": "",
  "phone": "",
  "linkedin": "",
  "skills": [],
  "education": [
    {{
      "degree": "",
      "institution": "",
      "year": ""
    }}
  ],
  "experience": [
    {{
      "company": "",
      "title": "",
      "duration": "",
      "description": ""
    }}
  ],
  "certifications": [],
  "projects": []
}}

Important Instructions:
- Return null for any missing fields.
- Normalize all date formats to "Month YYYY - Month YYYY" if possible.
- Only output valid JSON (no extra text, no explanations).
- Keep field names exactly same.

Resume Text ends here.
"""

    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    content = response['message']['content']
    return content

# Step 3: Parse the resume
def parse_resume(file_path):
    text = extract_text(file_path)
    raw_response = query_llm(text)

    try:
        parsed_data = json.loads(raw_response)
    except json.JSONDecodeError:
        print("Failed to parse JSON. Here is the raw response:")
        print(raw_response)
        parsed_data = None

    # Add confidence scores manually
    if parsed_data:
        parsed_data["confidence"] = {
            "name": 0.95 if parsed_data.get("name") else 0.0,
            "email": 1.0 if parsed_data.get("email") else 0.0,
            "phone": 1.0 if parsed_data.get("phone") else 0.0,
            "linkedin": 0.9 if parsed_data.get("linkedin") else 0.0,
            "skills": 0.95 if parsed_data.get("skills") else 0.0,
            "education": 0.9 if parsed_data.get("education") else 0.0,
            "experience": 0.9 if parsed_data.get("experience") else 0.0,
            "certifications": 0.9 if parsed_data.get("certifications") else 0.0,
            "projects": 0.9 if parsed_data.get("projects") else 0.0
        }

    return parsed_data

# Step 4: Example usage
if __name__ == "__main__":
    file_path = r"C:\Users\91934\Desktop\MOVIES\Mugeshsuryapradeep__AIML ENGINEER..pdf" # <-- Update your path
    result = parse_resume(file_path)
    print(json.dumps(result, indent=2))


{
  "name": "MUGESH SURYA PRADEEP R",
  "email": "mukeshsuryapradeep2@mail.com",
  "phone": "9345563268",
  "linkedin": "linkedin.com",
  "skills": [
    "Python",
    "SQL",
    "Scikit-learn",
    "Model Training",
    "Hyperparameter Tuning",
    "Cross-validation",
    "TensorFlow",
    "PyTorch",
    "Transformers",
    "Hugging Face",
    "LangChain",
    "NLTK",
    "SpaCy",
    "ARIMA",
    "SARIMA",
    "Prophet",
    "LSTM",
    "Pandas",
    "NumPy",
    "Feature Engineering",
    "ETL Pipelines",
    "Text processing",
    "Power BI",
    "Matplotlib",
    "Seaborn",
    "Power BI",
    "Plotly",
    "Streamlit",
    "Gradio",
    "Selenium",
    "AWS",
    "CI/CD",
    "Model Deployment",
    "Git",
    "Pinecone",
    "FAISS",
    "ChromaDB",
    "Retrieval-Augmented Generation (RAG)"
  ],
  "education": [
    {
      "degree": "B.E. Electrical and Electronic Engineering",
      "institution": "Loyola-ICAM College of Engineering and Technology",
      "year": "Nov 2020 \u